In [1]:
# Storage SC

In [2]:
import logging

import smartpynector as sp
from utils import *

In [3]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("StorageSC")

In [4]:
# Constants
READ_URL = "https://graphdb.odissei.nl/repositories/MateuszTest"
WRITE_URL = "https://graphdb.odissei.nl/repositories/MateuszTest/statements"

THERMOSTAT_API_URL = "http://0.0.0.0:8001/thermostat"
PREFIXES = {
    "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
    "saref": "https://w3id.org/saref#",
    "xsd": "http://www.w3.org/2001/XMLSchema#",
    "ex": "http://example.org/",
}

MEAS_GRAPH_PATTERN = """?meas rdf:type saref:Measurement .
                        ?meas saref:hasValue ?temp .
                        ?meas saref:isMeasuredIn saref:TemperatureUnit .
                        ?meas saref:hasTimestamp ?timestamp .
                        ?meas saref:isMeasurementOf ?room_id .
                        ?meas saref:relatesToProperty saref:Temperature .
                        ?meas saref:measurementMadeBy ?device_id ."""

H_MEAS_GRAPH_PATTERN = """?meas rdf:type saref:Measurement .
                        ?meas saref:hasValue ?temp .
                        ?meas saref:isMeasuredIn saref:TemperatureUnit .
                        ?meas saref:hasTimestamp ?timestamp .
                        ?meas saref:isMeasurementOf ?room_id .
                        ?meas saref:relatesToProperty saref:Temperature .
                        ?meas saref:measurementMadeBy ?device_id .
                        ?meas ex:measuredAfter ?startTimestamp .
                        ?meas ex:measuredBefore ?endTimestamp ."""

In [5]:
def parse_response(response, start_timestamp, end_timestamp):
    vars = response["head"]["vars"]
    bindings = response["results"]["bindings"]
    result = []
    for binding in bindings:
        measurement = {}
        for var in vars:
            measurement[var] = f'"{binding[var]["value"]}"'
        measurement["startTimestamp"] = f'"{start_timestamp}"'
        measurement["endTimestamp"] = f'"{end_timestamp}"'
        result.append(measurement)
    return result

In [6]:
# Return measurements in datetime range from GraphDB
def handle_answer_measurements(query_bindings):
    for binding in query_bindings:
        logger.info(f"Answer query bindings: {binding}")

        start_timestamp = binding["startTimestamp"]
        end_timestamp = binding["endTimestamp"]
        sparql_query = f"""PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
                            PREFIX saref: <https://w3id.org/saref#>
                            PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

                            SELECT *

                            WHERE {{
                                ?meas rdf:type saref:Measurement .
                                ?meas saref:hasValue ?temp .
                                ?meas saref:isMeasuredIn saref:TemperatureUnit .
                                ?meas saref:hasTimestamp ?timestamp .
                                FILTER (xsd:dateTime(?timestamp) >= "{start_timestamp}"^^xsd:dateTime && xsd:dateTime(?timestamp) <= "{end_timestamp}"^^xsd:dateTime)
                                ?meas saref:isMeasurementOf ?room_id .
                                ?meas saref:relatesToProperty saref:Temperature .
                                ?meas saref:measurementMadeBy ?device_id .
                                }}"""
        # logger.info(sparql_query)
        res = sp.run_sparql_query(READ_URL, sparql_query)
        result = parse_response(res, start_timestamp, end_timestamp)
        # logger.info(res)
        logger.info(result)

    return result


def handle_react_measurements(bindings):
    sp.store_data_in_graphdb(
        graph_pattern=MEAS_GRAPH_PATTERN,
        binding_set=bindings,
        prefixes=PREFIXES,
        read_url=READ_URL,
        write_url=WRITE_URL,
    )
    for binding in bindings:
        logger.info(f"Saving measurement {binding['meas']}")
    return []

In [7]:
def start_storage_kb(kb_id, kb_name, kb_description, ke_endpoint):
    register_knowledge_base(kb_id, kb_name, kb_description, ke_endpoint)

    answer_measurements_ki = register_answer_knowledge_interaction(
        H_MEAS_GRAPH_PATTERN,
        "answer-historical-measurements",
        kb_id,
        ke_endpoint,
        PREFIXES,
    )

    react_measurements_ki = register_react_knowledge_interaction(
        MEAS_GRAPH_PATTERN,
        None,
        "react-measurements",
        kb_id,
        ke_endpoint,
        PREFIXES,
    )

    start_handle_loop(
        {
            answer_measurements_ki: handle_answer_measurements,
            react_measurements_ki: handle_react_measurements,
        },
        kb_id,
        ke_endpoint,
    )

In [8]:
start_storage_kb(
    "http://example.org/storage",
    "Storage",
    "GraphDB smart connector",
    "http://knowledge_engine:8280/rest/",
)

2023-06-21 16:00:14 INFO registered Storage
2023-06-21 16:00:14 INFO received issued knowledge interaction id: http://example.org/storage/interaction/answer-historical-measurements
2023-06-21 16:00:15 INFO received issued knowledge interaction id: http://example.org/storage/interaction/react-measurements
2023-06-21 16:00:41 INFO Answer query bindings: {'room_id': '"http://0.0.0.0:8001/thermostat/rooms/1"', 'temp': 'n/a', 'device_id': '"http://0.0.0.0:8001/thermostat/devices/1"', 'meas': '<http://0.0.0.0:8001/thermostat/measurements/fbc689c8-0f9f-4465-8aa0-a6caab1c0c5a>', 'endTimestamp': '2023-06-21T18:10:00+00:00', 'startTimestamp': '2023-06-20T18:00:00+00:00', 'timestamp': '"2023-06-21T16:00:41+00:00"'}
2023-06-21 16:00:42 INFO [{'meas': '"http://0.0.0.0:8001/thermostat/measurements/fef25f7d-025c-4c58-8acb-51bc19427f41"', 'temp': '"9"', 'timestamp': '"2023-06-20T19:15:03+00:00"', 'room_id': '"http://0.0.0.0:8001/thermostat/rooms/1"', 'device_id': '"http://0.0.0.0:8001/thermostat/devic

KeyboardInterrupt: 

In [ ]:
# java.util.concurrent.CompletionException: java.lang.IllegalArgumentException: KB gave outgoing binding Binding [map={room_id="http://0.0.0.0:8001/thermostat/rooms/1", temp="13", device_id="http://0.0.0.0:8001/thermostat/devices/1", meas="http://0.0.0.0:8001/thermostat/measurements/07ade4c8-8221-4679-9348-0972cae20ad8", endTimestamp="2023-06-21T15:28:09+00:00", startTimestamp="2023-06-20T21:00:00+00:00", timestamp="2023-06-20T21:56:34+00:00"}], but this doesn't have a matching incoming binding!